In [1]:
import pandas as pd

df = pd.read_csv('/Users/matthiasgalka/git/ppchem_project/data/AfterRFfilter(1).csv')

In [3]:
df.head()
df.shape

(9144, 4)

In [4]:
import re

#Rf_check = r'( ?R[fF]?[ :=˜(]?)'
Rf_check = r'( ?(R|r)[fF][ :=˜0(])'

rows_to_drop = []

for index, row in df.iterrows():
    checkRf = re.findall(Rf_check, row['paragraphText'])
    if not checkRf:
        rows_to_drop.append(index)

In [6]:
len(rows_to_drop)

569

In [6]:
#dropping all compounds without a Rf value
df_filtered = df.drop(rows_to_drop)
df_filtered.shape
#df_filtered1.head(10)
#df_short = df_filtered[:10] #making a small test dataframe
#df_short.reset_index(drop=True, inplace=True)
#df_short.shape

(35634, 4)

In [8]:
#df.loc[701, 'paragraphText'].apply(lambda x: re.findall(Rf_check, x))
#df.loc[164, 'paragraphText']
df.loc[161, 'paragraphText']

'0.25 g of 4-(3,4-dihydro-2H-quinolin-1-yl)-6-iodoquinazoline, 0.30 g of 2-methyl-6-(4,4,5,5-tetramethyl-1,3,2-dioxaborolan-2-yl)-3-(2-trimethylsilanyl-ethoxymethyl)-3H-imidazo[4,5-b]pyridine, 0.16 g of sodium hydrogencarbonate and 0.09 g of Pd(PPh3)2Cl2 in 5.00 ml of dioxane and 0.50 ml of water are heated at 90° C. under nitrogen in a flask until the reaction is complete (HPLC check, about 5 hours). The cooled reaction solution is diluted with EA and washed 3 times with water. The organic phase is dried over sodium sulfate and purified by means of column chromatography (gradient EA: methanol 0-40% in 20 min), giving 0.19 g of 4-(3,4-dihydro-2H-quinolin-1-yl)-6-[2-methyl-3-(2-trimethylsilanylethoxymethyl)-3H-imidazo[4,5-b]pyridin-6-yl]quinazoline as white powder (yield 50%, content 94%); MS-FAB (M+H+)=523.2; Rf (polar method): 2.54 min.'

In [16]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
#client = Groq(api_key='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')

instruction = 'extract the Rf values, the used for it and the solvent ratio from the following text:'
format = 'give me the data in the following format: Rf = x.xx  SolventA = nameA  SolventB = nameB  Solventratio: x:x (The first number should be the part of SolventA)'
no_value = 'if you can not find a clear sign of Rf values or solvent informations just give me back: "!x!"'

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"{instruction} {df.loc[161, 'paragraphText']} {format} {no_value}",
        }
    ],
    model="mixtral-8x7b-32768",
)
print(chat_completion.choices[0].message.content)

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [18]:
def LLM_find_values(text):

    client = Groq(api_key='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')

    instruction = 'extract the Rf values, the used for it and the solvent ratio from the following text:'
    format = 'give me the data in the following format: Rf = x.xx  SolventA = AnameA  SolventB = BnameB  Solventratio: x:x (The first number should be the part of SolventA)'
    no_value = 'if you can not find a clear sign of Rf values or solvent informations just give me back: "!x!"'

    chat_completion = client.chat.completions.create(
        messages=[
            {   
                "role": "user",
                "content": f"{instruction} {text} {format} {no_value}",
            }
        ],
        model="mixtral-8x7b-32768",
    )
    return chat_completion.choices[0].message.content



In [20]:
LLM_find_values(df_filtered.loc[25, 'paragraphText'])

'Based on the provided text, the extraction yields the compound ethyl (1-benzyl-piperidin-4-ylidene)cyanoacetate with an Rf value of 0.53 in a solvent mixture of hexane and acetone oxide (AcOEt) with a ratio of 1:1. Here is the data in the requested format:\n\nRf = 0.53\nSolventA = hexane\nSolventB = AcOEt\nSolventratio: 1:1'

In [196]:
df_short['raw_data'] = df_short['paragraphText'].apply(lambda x: LLM_find_values(x))

/var/folders/56/k5y5cj453pvcsblxlzlzn0340000gn/T/ipykernel_11161/4179265979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['raw_data'] = df_short['paragraphText'].apply(lambda x: LLM_find_values(x))


In [200]:
df_short.head(10)

,paragraphText,reactionSmiles,productSmiles,title,raw_data
0,[A] Synthesis of 1-benzyl-piperidin-4-ylidene)...,C(CC([O-])=O)#N.[CH2:7]([N:14]1[CH2:19][CH2:18...,['C(C1=CC=CC=C1)N1CCC(CC1)=C(C(=O)OCC)C#N'],NaN,Rf = 0.53\nSolventA = hexane\nSolventB = AcOEt...
1,A solution of (S)-(+)-3-hydroxytetrahydrofuran...,O[C@H]1CCOC1.CC([O-])(C)C.[K+].[CH3:13][O:14][...,['COC(=O)C=1SC=CC1'],NaN,"Based on the given text, the Rf value, the sol..."
2,A solution of 4-{[(phenylmethoxy)carbonylamino...,[C:1]1([CH2:7][O:8][C:9]([NH:11][CH2:12][C:13]...,['CON(C(=O)C1=CC=C(C=C1)CNC(=O)OCC1=CC=CC=C1)C'],Preparation of N-methoxy-N-methyl(4-{[(phenylm...,Rf = 0.3\nSolventA = hexanes\nSolventB = EtOAc...
3,Putative nitrilase up-mutants were assayed in ...,[CH3:1][C:2]1(C)S[C@@H]2[C@H](NC([C@H](N)C3C=C...,['O[C@@H](CC(=O)O)CC#N'],NaN,"Based on the text, the Rf value, the solvent u..."
4,3.0 g N6-Benzoyl-5′-O-tert-butyldimethylsilyl-...,[C:1]([NH:9][C:10]1[C:11]2[N:12]=[CH:13][N:14]...,['C(C1=CC=CC=C1)(=O)NC=1C=2N=CN([C@H]3C[C@H](O...,NaN,Rf = 0.6\nSolventA = EtOAc\nSolventB = hex\nSo...
5,To 0.4 g N6-benzoyl-3′-O-(methylthiomethyl)-5′...,[C:1]([NH:9][C:10]1[C:11]2[N:12]=[CH:13][N:14]...,['C(C1=CC=CC=C1)(=O)NC=1C=2N=CN([C@H]3C[C@H](O...,NaN,Rf = 0.3\nSolventA = EtOAC\nSolventB = Hex\nSo...
6,3.5 g N4-benzoyl-5′-O-tert-butyldimethylsilyl-...,[C:1]([NH:9][C:10]1[CH:30]=[CH:29][N:13]([C@@H...,['C(C1=CC=CC=C1)(=O)NC1=NC(N([C@H]2C[C@H](OCSC...,NaN,Rf = 0.4 SolventA = EtOAc SolventB = hex Solve...
7,To 0.5580 g N4-benzoyl-3′-O-(methylthiomethyl)...,[C:1]([NH:9][C:10]1[CH:33]=[CH:32][N:13]([C@@H...,['C(C1=CC=CC=C1)(=O)NC1=NC(N([C@H]2C[C@H](OCN=...,NaN,"Based on the given text, the Rf value, the sol..."
8,To 786 mg 12 (1.1 mmol) dissolved in 8 mL dry ...,[C:1]([NH:6][C:7]1[N:8]=[C:9]([O:34][C:35](=[O...,['C(C(C)C)(=O)NC=1N=C(C=2N=CN([C@H]3C[C@H](OCN...,NaN,Rf = 0.3\nSolventA = Hex (hexane)\nSolventB = ...
9,"Methyl (S)-2-[(2S,3S)-2-((3S,4S)-3-benzyloxyca...",[CH2:1]([O:8][C:9]([NH:11][C@H:12]([C@@H:33]([...,['N[C@H]([C@H](CC(=O)N[C@H](C(=O)N[C@H](C(=O)O...,NaN,"Based on the text provided, the following info..."


In [24]:
df_short.loc[0, 'paragraphText']

'[A] Synthesis of 1-benzyl-piperidin-4-ylidene)-cyanoacetate (2; FIG. 4). A solution of 1-benzyl-piperidin-4-one (75.1 g, 0.40 mol) in toluene (9400 mL), ethyl cyanoacetate (50.6 mL, 0.48 mol) and acetic acid (918.2 mL, 0.32 mol) was refluxed for 4 hr. The mixture was quenched with ice-water and extracted with diethyl ether. The combined extracts were washed with water, brine and dried over Na2SO4 to give ethyl (1-benzyl-piperidin-4-ylidene)cyanoacetate in quantitative yield. Rf=0.53 (hexane:AcOEt=1:1).'

In [216]:
pattern_rf = r'\d\.\d\d?'

df_short['Rf_value'] = df_short['raw_data'].apply(lambda x: float(re.findall(pattern, x)[0]))
# searching for pattern in raw_data and takes the first value in list and converts it to a float

/var/folders/56/k5y5cj453pvcsblxlzlzn0340000gn/T/ipykernel_11161/2536720413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['Rf_value'] = df_short['raw_data'].apply(lambda x: float(re.findall(pattern, x)[0]))


In [ ]:
pattern_ratio

df_short['Rf_value'] = df_short['raw_data'].apply(lambda x: float(re.findall(pattern_ratio, x)[0]))

In [217]:
df_short.head()

,paragraphText,reactionSmiles,productSmiles,title,raw_data,Rf_value
0,[A] Synthesis of 1-benzyl-piperidin-4-ylidene)...,C(CC([O-])=O)#N.[CH2:7]([N:14]1[CH2:19][CH2:18...,['C(C1=CC=CC=C1)N1CCC(CC1)=C(C(=O)OCC)C#N'],NaN,Rf = 0.53\nSolventA = hexane\nSolventB = AcOEt...,0.53
1,A solution of (S)-(+)-3-hydroxytetrahydrofuran...,O[C@H]1CCOC1.CC([O-])(C)C.[K+].[CH3:13][O:14][...,['COC(=O)C=1SC=CC1'],NaN,"Based on the given text, the Rf value, the sol...",0.44
2,A solution of 4-{[(phenylmethoxy)carbonylamino...,[C:1]1([CH2:7][O:8][C:9]([NH:11][CH2:12][C:13]...,['CON(C(=O)C1=CC=C(C=C1)CNC(=O)OCC1=CC=CC=C1)C'],Preparation of N-methoxy-N-methyl(4-{[(phenylm...,Rf = 0.3\nSolventA = hexanes\nSolventB = EtOAc...,0.30
3,Putative nitrilase up-mutants were assayed in ...,[CH3:1][C:2]1(C)S[C@@H]2[C@H](NC([C@H](N)C3C=C...,['O[C@@H](CC(=O)O)CC#N'],NaN,"Based on the text, the Rf value, the solvent u...",0.50
4,3.0 g N6-Benzoyl-5′-O-tert-butyldimethylsilyl-...,[C:1]([NH:9][C:10]1[C:11]2[N:12]=[CH:13][N:14]...,['C(C1=CC=CC=C1)(=O)NC=1C=2N=CN([C@H]3C[C@H](O...,NaN,Rf = 0.6\nSolventA = EtOAc\nSolventB = hex\nSo...,0.60


In [1]:
pattern_solventA = r'SolventA = (\w+)'
pattern_solventB = r'SolventB = (\w+)'

df_short['SolventA'] = df_short['raw_data'].apply(lambda x: str(re.findall(pattern_solventA, x)))
df_short['SolventB'] = df_short['raw_data'].apply(lambda x: str(re.findall(pattern_solventB, x)))

NameError: name 'df_short' is not defined

In [229]:
df_short.head()

,paragraphText,reactionSmiles,productSmiles,title,raw_data,Rf_value,SolventA
0,[A] Synthesis of 1-benzyl-piperidin-4-ylidene)...,C(CC([O-])=O)#N.[CH2:7]([N:14]1[CH2:19][CH2:18...,['C(C1=CC=CC=C1)N1CCC(CC1)=C(C(=O)OCC)C#N'],NaN,Rf = 0.53\nSolventA = hexane\nSolventB = AcOEt...,0.53,['hexane']
1,A solution of (S)-(+)-3-hydroxytetrahydrofuran...,O[C@H]1CCOC1.CC([O-])(C)C.[K+].[CH3:13][O:14][...,['COC(=O)C=1SC=CC1'],NaN,"Based on the given text, the Rf value, the sol...",0.44,['dichloromethane']
2,A solution of 4-{[(phenylmethoxy)carbonylamino...,[C:1]1([CH2:7][O:8][C:9]([NH:11][CH2:12][C:13]...,['CON(C(=O)C1=CC=C(C=C1)CNC(=O)OCC1=CC=CC=C1)C'],Preparation of N-methoxy-N-methyl(4-{[(phenylm...,Rf = 0.3\nSolventA = hexanes\nSolventB = EtOAc...,0.30,['hexanes']
3,Putative nitrilase up-mutants were assayed in ...,[CH3:1][C:2]1(C)S[C@@H]2[C@H](NC([C@H](N)C3C=C...,['O[C@@H](CC(=O)O)CC#N'],NaN,"Based on the text, the Rf value, the solvent u...",0.50,['EtOAc']
4,3.0 g N6-Benzoyl-5′-O-tert-butyldimethylsilyl-...,[C:1]([NH:9][C:10]1[C:11]2[N:12]=[CH:13][N:14]...,['C(C1=CC=CC=C1)(=O)NC=1C=2N=CN([C@H]3C[C@H](O...,NaN,Rf = 0.6\nSolventA = EtOAc\nSolventB = hex\nSo...,0.60,['EtOAc']


In [26]:
df_short.loc[1, 'paragraphText']

'A solution of (S)-(+)-3-hydroxytetrahydrofuran (259 mg, 2.94 mmol) in dry NMP (1 mL) was added to a solution of KOtBu (271 mg, 2.42 mmol) in dry NMP (1 mL). The reaction was stirred at room temperature for 15 min, then a solution of 5-(3,3-dimethyl-but-1-ynyl)-3-(1-oxa-spiro[2.5]oct-6-ylamino]-thiophene-2-carboxylic acid methyl ester (200 mg, 0.576 mmol) in dry NMP (4.0 mL) was added and the reaction heated in a 40° C. oil bath for 24 h. The reaction was poured into ice water, cooled to 0° C., neutralized with 5% citric acid to pH=5-6 and extracted with ethyl acetate. The organic layer was washed with 5% LiCl, brine, dried and concentrated to give a brown oil which was taken up in dichloromelhane/MeOH (5.0 mL/1.0 mL) and treated with the dropwise addition of TMSCH2N2 (0.35 mL, 0.692 mmol, 2M in hexane). After 20 min, volatiles were removed under vacuum and the crude residue was purified by column chromatography on silica gel (Teledyne Isco Redisep Rf Gold™) eluting with 100% dichlorom